In [ ]:
%autosave 60
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import json
import os
import pickle
from collections import Counter, OrderedDict, defaultdict
from copy import deepcopy
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union, cast

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL.Image as pil_img
import seaborn as sns
import sklearn as skl
from IPython.display import Image, display
from matplotlib.patches import Rectangle
from matplotlib_inline.backend_inline import set_matplotlib_formats
from tqdm.contrib import tenumerate, tmap, tzip
from tqdm.contrib.bells import tqdm, trange

from geoscreens.consts import (
    EXTRACTED_FRAMES_PATH,
    FRAMES_METADATA_PATH,
    LATEST_DETECTION_MODEL_NAME,
    VIDEO_PATH,
)
from geoscreens.data import get_all_geoguessr_split_metadata
from geoscreens.data.metadata import GOOGLE_SHEET_IDS, FramesList
from geoscreens.utils import batchify, load_json, save_json, timeit_context

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 15)
pd.set_option("display.max_rows", 50)
# Suitable default display for floats
pd.options.display.float_format = "{:,.2f}".format
plt.rcParams["figure.figsize"] = (12, 10)

# This one is optional -- change graphs to SVG only use if you don't have a
# lot of points/lines in your graphs. Can also just use ['retina'] if you
# don't want SVG.
%config InlineBackend.figure_formats = ["retina"]
set_matplotlib_formats("pdf", "png")

* [x] Load in_game frames
* [x] load detections for all videos
* [x] filter to in_game frames
* [ ] crop images
* [ ] ocr cropped images
* [ ] save results

## Functions

## Load in_game Frames

In [ ]:
df_ingame = pickle.load(open("/shared/gbiamby/geo/segment/in_game_frames_000.pkl", "rb"))

In [ ]:
df_url_frames = df_ingame[df_ingame.labels.apply(lambda l: "url" in l)].copy(deep=True)

In [ ]:
print(df_ingame.shape, df_url_frames.shape)

In [ ]:
df_ingame.video_id.nunique(), df_url_frames.video_id.nunique()

In [ ]:
df_url_frames.head(1).T

In [ ]:
import operator

import easyocr


def last_index(lst, value):
    return len(lst) - operator.indexOf(reversed(lst), value) - 1


reader = easyocr.Reader(["en"])

In [ ]:
urls = defaultdict(list)

for i, (idx, row) in tenumerate(df_url_frames.iterrows(), total=len(df_url_frames)):
    # if i >= 100:
    #     break
    # print(row)
    video_id = row.video_id
    url_idx = last_index(row.labels, "url")
    # Crop:
    img = pil_img.open(row.file_path)
    # display(img)
    url_area = row.bboxes[url_idx]
    url_area = (url_area["xmin"], url_area["ymin"], url_area["xmax"], url_area["ymax"])
    img_cropped = img.crop(url_area)
    # display(img_cropped)
    result = reader.recognize(np.array(img_cropped))
    urls[video_id].append({**row.to_dict(), "ocr": result})
    # print(result)

### Show the cropped URL bar

In [ ]:
display(img_cropped)

### OCR on the cropped URL bar

In [ ]:
result = reader.recognize(np.array(img_cropped))
result

In [ ]:
df_url_frames["row_num"] = df_url_frames.reset_index().index
df_url_frames["gpu_id"] = df_url_frames.row_num.apply(lambda x: x % 3)

In [ ]:
df_url_frames["gpu_id"] = df_url_frames.row_num.apply(lambda x: x % 3)

In [ ]:
results = pickle.load(open("/shared/gbiamby/geo/data/urls/url_ocr_raw.pkl", "rb"))

In [ ]:
results

---

## Show First Frames of Random Rounds & Games

In [ ]:
from IPython.core.display import HTML, Markdown
from PIL import ImageDraw


def transform_box(x1, y1, x2, y2, target_width, target_height, curr_dim=640):
    """
    Transform bbox coordinates from (curr_dim, curr_dim) pixel space to size=(width, height) pixel
    space. assumes width is greater than height. This is used because the detector bbox coordinates
    are in a square pixel space (config.dataset_config.img_size)**2, and we need to convert the bbox
    coordinates back to the original image pixel space (e.g., 1280*720).

    Args:
        xmin, ymin, xmax, ymax

    Returns:
        Tuple[[xmin, ymin, xmax, ymax], area]
    """
    # Back to width*width:
    new_x1 = x1 * (target_width / curr_dim)
    new_y1 = y1 * (target_width / curr_dim)
    new_x2 = x2 * (target_width / curr_dim)
    new_y2 = y2 * (target_width / curr_dim)
    # Remove vertical padding
    y_pad = (target_width - target_height) / 2
    new_y1 -= y_pad
    new_y2 -= y_pad
    new_area = (new_x2 - new_x1 + 1) * (new_y2 - new_y1 + 1)
    return (new_x1, new_y1, new_x2, new_y2), new_area


def get_dets(video_id: str, model: str, df_meta: pd.DataFrame):
    split = df_meta.loc[video_id].split
    dets_path = Path(
        f"/shared/gbiamby/geo/segment/detections/{model}/{split}/df_frame_dets-video_id_{video_id}.pkl"
    )
    df_dets = pickle.load(open(dets_path, "rb"))
    if "frame_id" in df_dets.columns:
        df_dets.drop(columns=["frame_id"], inplace=True)
    df_dets.set_index("frame_idx", inplace=True)

    # df_dets.bbox.apply(lambda x: transform_box(*x.values(),
    return df_dets


def show_random_frames_masked(
    video_id: str, model: str, df: pd.DataFrame, df_meta: pd.DataFrame, n_samples: int = 5
):
    df_random = df.sample(n=n_samples)
    df_dets = get_dets(video_id, model, df_meta)
    for idx, img_row in df_random.iterrows():
        print("-" * 180)
        print(
            f"video_id: {img_row.video_id}, frame_idx: {img_row.frame_idx}, seconds: {img_row.sec}",
        )
        img = pil_img.open(img_row["file_path"])
        # img.thumbnail((1080, 640), pil_img.NEAREST)
        img_width, img_height = img.size
        display(img)
        dets = df_dets.loc[img_row.frame_idx]
        # display(dets)
        dets_lookup = {
            l: (l, transform_box(*bb.values(), img_width, img_height), s)
            for l, bb, s in zip(dets.labels, dets.bboxes, dets.scores)
        }
        # print(dets_lookup)
        masked_area = sum([d[1][1] for d in dets_lookup.values()])
        print(
            f"masked_area: {masked_area:,}",
            f"img_area: {float(img_width*img_height):,}",
            f"pct_masked: {100.0 * masked_area / (img_width*img_height):.2f}%",
        )

        img_masked = img
        draw = ImageDraw.Draw(img_masked)
        for label, bbox, score in dets_lookup.values():
            draw.rectangle(bbox[0], fill=0)

        # Mask out minimum rectangular region that encloses the geoguessr logo and/or the status bar:
        top_ui = [dets_lookup[l] for l in ["game_title", "status_bar"] if l in dets_lookup]
        if top_ui:
            y_max = max(d[1][0][3] for d in top_ui)
            # xmin, ymax = reverse_point(640, y_max, img_width, img_height, 640)
            draw.rectangle((0, 0, img_width, y_max), fill=0)
        display(img_masked)
        print("")


if "df_meta" not in locals():
    df_meta = pd.DataFrame(get_all_geoguessr_split_metadata().values()).set_index("id")
if "frame_paths" not in locals():
    frame_paths = pickle.load(open(EXTRACTED_FRAMES_PATH / "frames_list.pkl", "rb"))


video_id = "--0Kbpo9DtE"
video_id = "zOoUR17xnL0"
model = LATEST_DETECTION_MODEL_NAME
# frames = subsample_frames(video_id, df_frames_meta, frame_paths)
# df_all_frames = pd.DataFrame(frames)
# in_game_frames = filter_to_in_game(video_id, frames, df_meta)
# df_ingame = pd.DataFrame(in_game_frames).sort_values(["round_num", "frame_idx"])

show_random_frames_masked(video_id, model, df_ingame.loc[video_id], df_meta, 10)

---

## Use Initial Frame(s) To Identify Rounds

In [ ]:
def get_first_name_frames(
    video_id: str, df_ingame: pd.DataFrame, n_samples: int = 5, round_num: int = None
):
    """
    Get first n_samples frames from given round of the video
    """
    video_rows = df_ingame.loc[video_id].sort_index()
    results = []
    rounds = set(video_rows.round_num.unique().tolist())
    if round_num:
        rounds = rounds.intersection(set([round_num]))
    for round_num in rounds:
        start, end = -1, -1
        total_sampled = 0
        round_rows = video_rows[video_rows.round_num == round_num]
        for i, (idx, row) in enumerate(round_rows.iterrows()):
            # if i<=6:
            #     continue
            if start < 0:
                if True or "in_game_mini_map" in row.labels:
                    start = row.frame_idx
                else:
                    continue
            total_sampled += 1
            end = row.frame_idx
            if total_sampled > n_samples:
                break
            print(row)
            img = display(pil_img.open(row.file_path))

        print(f"start: {start}, end: {end}")
        print(len(round_rows.loc[start:end]))
        results.append(round_rows.loc[start:end])
    return results


video_id = "zOoUR17xnL0"
video_id = "--0Kbpo9DtE"
results = get_first_name_frames(video_id, df_ingame, n_samples=5, round_num=8)

In [ ]:
video_rows = df_ingame.loc[video_id].sort_index()
video_rows.shape
video_rows.round_num.unique().tolist()

---

## Load Raw OCR Results, Clean up the URLs and Group Them by video_id + game_num

In [ ]:
ocr = pickle.load(open("/shared/gbiamby/geo/data/urls/url_ocr_raw.pkl", "rb"))

In [ ]:
ocr["--0Kbpo9DtE"][0]

In [ ]:
# How many ocr outputs have more than one result?

# rawr = []
# for i, (video_id, frames) in tenumerate(ocr.items()):
#     for f in frames:
#         if len(f["ocr"]) > 1:
#             rawr.append({"video_id": video_id, "ocr": f["ocr"]})
# print(len(rawr))

In [ ]:
ocr_clean = []
for i, (video_id, frames) in tenumerate(ocr.items()):
    for f in frames:
        for ocr_result in f["ocr"]:
            ocr_clean.append(
                {
                    "video_id": video_id,
                    "ocr": ocr_result[1],
                    "file_path": f["file_path"],
                    "round_num": f["round_num"],
                }
            )

In [ ]:
len(ocr_clean)

In [ ]:
df_ocr = pd.DataFrame(ocr_clean)
df_ocr.ocr = (
    df_ocr.ocr.astype("string")
    .str.replace("\s\s+", " ")
    .str.replace("^[0-9]*\s*[l|]*", "", regex=True)
    .str.replace("| |", "||", regex=False)
    .str.replace("https||", "", regex=False)
    .str.replace("https|", "", regex=False)
    .str.replace("https[|l]{0,2}", "")
    .str.replace("Secure |", "", regex=False)
    .str.replace("Secure", "")
    .str.replace("||", "", regex=False)
    .str.replace("Il", "", regex=False)
    .str.replace("ssrcon", "ssr.com", regex=False)
    .str.replace("con/", "com/", regex=False)
    .str.replace("cor/", "com/", regex=False)
    .str.replace("[.\s]*c[cao0][mnr]\s*/", ".com/")
    .str.replace("[.\s]*c[cao0][mnr]\s*", ".com")
    .str.replace(".*?eoguessr", "geoguessr")
    .str.replace("g.*?oguessr", "geoguessr")
    .str.replace("ge.*?guessr", "geoguessr")
    .str.replace("geo.*?uessr", "geoguessr")
    .str.replace("geog.*?essr", "geoguessr")
    .str.replace("geogues.*?r", "geoguessr")
    .str.replace("geoguess.*?", "geoguessr")
    .str.replace("geoguessrr", "geoguessr")
    # Two
    .str.replace("..oguessr", "geoguessr")
    .str.replace(".e.guessr", "geoguessr")
    .str.replace(".eo.uessr", "geoguessr")
    .str.replace(".eoguess.", "geoguessr")
    .str.replace("g.*?oguess.*?", "geoguessr")
    .str.replace("g.*?o.?uess.*?", "geoguessr")
    # two
    .str.replace("g.*?.*?guessr", "geoguessr")
    .str.replace("g.{0,3}?uessr", "geoguessr")
    .str.replace("g.*?o.uessr", "geoguessr")
    .str.replace("g.*?oguess.", "geoguessr")
    .str.replace(".+?eo.+?uess[a-zA-Z]{1}", "geoguessr")
    .str.replace("ld\s*/?\s*play", ".com/play")
    .str.replace("^eog", "geog")
    .str.replace("geoguessr[^.]+?co", "geoguessr.co")
    # .str.replace("geoguess co[mnr]", "geoguessr.com")
    # .str.replace("geoguessco[mnr]", "geoguessr.com")
    # .str.replace("geoguessrco[mnr]", "geoguessr.com")
    .str.replace("geoguessr.*com", "geoguessr.com")
    .str.replace("geoguessr\s*o[mnr]", "geoguessr.com")
    # Strip text before "geoguess"
    .str.replace("^.+(?=geoguess)", "")
    #
    .str.replace("geoguessr.*\.com", "geoguessr.com")
    #
    .str.replace("geoguessr\.com.*challenge", "geoguessr.com/challenge")
    .str.replace("geoguessr\.com/challenge[^/]{1}", "geoguessr.com/challenge/")
    .str.replace("geoguessr\.com.*play", "geoguessr.com/play")
    .str.replace("geoguessr\.com/play[^/]{1}", "geoguessr.com/play/")
    .str.replace("(?<=.)?comuk[/]?", "com/uk/")
    .str.replace("/.{1,2}lay", "/play")
    .str.strip()
)
exclude = set(["did you enjoy", "channel v"])
for e in exclude:
    df_ocr = df_ocr[~(df_ocr.ocr.str.lower().str.contains(e))].copy(deep=True)
df_ocr["url_count"] = df_ocr.merge(df_ocr.groupby("ocr").count(), on="ocr")[["video_id_y"]]
df_ocr[~df_ocr.ocr.str.contains("geoguessr.com")].sort_values("url_count")

In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        pd.DataFrame(df_ocr[~df_ocr.ocr.str.contains("geoguessr.com")].ocr.value_counts()).head()
    )

In [ ]:
exclude = set(["geoguessr.com/play"])
df_clean = df_ocr[
    (df_ocr.ocr.str.contains("geoguessr.com"))
    & ~(df_ocr.ocr.str.contains("retro"))
    & ~(df_ocr.ocr.isin(exclude))
].copy(deep=True)
df_clean["slug"] = (
    df_clean.ocr.str.replace("geoguessr.com", "")
    .str.replace("challenge", "")
    .str.replace("play", "")
    .str.replace("/", "")
)
df_clean["game_num"] = df_clean.round_num.apply(lambda rn: rn // 5)
df_clean["slug_len"] = df_clean.slug.apply(lambda s: len(s))
df_clean = df_clean[(df_clean.slug_len > 10) & (df_clean.slug_len < 80)].copy(deep=True)
df_clean.sort_values("slug_len")

In [ ]:
df_clean2 = (
    pd.DataFrame(
        df_clean.groupby(["video_id", "game_num", "ocr"]).agg(
            url_count=("ocr", "count"),
            file_path=("file_path", "max"),
        )
    )
    .reset_index()
    .sort_values(["video_id", "game_num", "url_count"], ascending=[True, True, False])
)
df_clean2["ocr_rank"] = (
    df_clean2.groupby(["video_id", "game_num"])["url_count"]
    .transform(lambda x: x.rank(method="first", ascending=False))
    .astype("int")
)
df_clean2 = df_clean2[["video_id", "game_num", "ocr", "ocr_rank", "url_count", "file_path"]]

In [ ]:
print("Total video_ids: ", df_clean2.video_id.nunique())
print("Total games: ", len(df_clean2.groupby(["video_id", "game_num"]).count()))

In [ ]:
df_clean2

In [ ]:
print(
    "Number of video_id's with URL detections from OCR, that are not in the google sheet: ",
    len(set(df_clean2.video_id.values.tolist()) - (set(GOOGLE_SHEET_IDS))),
)

In [ ]:
if True:
    pickle.dump(df_clean2, open("/shared/gbiamby/geo/data/urls/url_ocrs_cleaned.pkl", "wb"))
    df_clean2.to_csv("/shared/gbiamby/geo/data/urls/url_ocrs_cleaned.csv", index=False, header=True)

In [ ]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     # display(pd.DataFrame(df_clean[(df_clean.slug_len < 8)].ocr.value_counts()))
#     display(pd.DataFrame(df_clean[(df_clean.slug_len > 0)].ocr.value_counts()))

In [ ]:
df_clean.slug_len.plot.hist(bins=50)